In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# enter "dat" directory, enter all subdirectories with name ending in "1","2","3" and load all csv files
# with name n into a list of dataframes
def load_data(n: str):
    data = []
    for root, dirs, files in os.walk("dat"):
        for dir in dirs:
            if dir[-1] in ["1","2","3"]:
                for file in os.listdir(os.path.join(root,dir)):
                    if file == n:
                        data.append(pd.read_csv(os.path.join(root,dir,file)))
    return data

In [12]:
populations = load_data("population.csv")
experiment_data = load_data("experiment.csv")

90